**ATTENTION!!!**
BE SURE that your runtime is GPU.

In [ ]:
# if you are using this script on server you can specify th visible free cuda device
# the memory usage of devices and their id can be checked with 'nvidia-smi' or 'nvidia-cdl' commands 

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '2' # here the id of the device

### Cloning repo from git

This operation requires that you have access to the repo [paradetox](https://github.com/skoltech-nlp/paradetox/).

Generate (or use already existing) your GitHub access token. 

If you need to generate new one, you can use the [instruction](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token).

In [1]:
!git clone https://Rookie20001122@github.com/skoltech-nlp/paradetox.git

fatal: destination path 'paradetox' already exists and is not an empty directory.


### Installing required libs

In [17]:
!pip install transformers
!pip install python-box
!pip install flair
!pip install sentencepiece
!pip install sacremoses

In [18]:
!pip install fairseq

### Preparing input and args

In [19]:
from box import Box

In [3]:
args = Box({
    'batch_size': 32,
    'cola_classifier_path': '/content/drive/MyDrive/style_transfer',
    'wieting_tokenizer_path': 'sim.sp.30k.model',
    'wieting_model_path': 'sim.pt',
    't1': 75., # this is default value
    't2': 70., # this is default value
    't3': 12. # this is default value
})

In [36]:
# test example
# TODO: load you data here

preds = [
    'you are fucker',
    'you are fucker',
    'you are fucker'
]

inputs = [
    'you are motherfucker',
    'you is very amazing',
    'you are fucker',
]

### Style Transfer Accuracy metric

In [6]:
from paradetox.evaluation_detox.metric_tools.style_transfer_accuracy import classify_preds
import numpy as np

In [7]:
accuracy_by_sent = classify_preds(args, preds)
accuracy = np.mean(accuracy_by_sent)
print('\n')
print(accuracy)

Calculating style of predictions


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1/1 [00:00<00:00,  4.51it/s]



0.0


### Content Similarity metrics

Aditional data download

In [8]:
!wget https://storage.yandexcloud.net/nlp/wieting_similarity_data.zip
!unzip wieting_similarity_data.zip

--2023-04-06 22:53:15--  https://storage.yandexcloud.net/nlp/wieting_similarity_data.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74066204 (71M) [application/x-zip-compressed]
Saving to: ‘wieting_similarity_data.zip’

wieting_similarity_ 100%[===================>]  70.63M  20.2MB/s    in 4.4s    

2023-04-06 22:53:20 (16.0 MB/s) - ‘wieting_similarity_data.zip’ saved [74066204/74066204]

Archive:  wieting_similarity_data.zip
  inflating: sim.pt                  
  inflating: sim.sp.30k.model        


In [9]:
from paradetox.evaluation_detox.metric_tools.content_similarity import calc_bleu
from paradetox.evaluation_detox.metric_tools.content_similarity import flair_sim
from paradetox.evaluation_detox.metric_tools.content_similarity import wieting_sim

In [ ]:
bleu = calc_bleu(inputs, preds)
print('\n')
print(bleu)

Calculating BLEU similarity


In [ ]:
emb_sim_stats = flair_sim(args, inputs, preds)
emb_sim = emb_sim_stats.mean()
print('\n')
print(emb_sim)

Calculating flair embeddings similarity
2021-10-12 11:48:52,917 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmpgur_rqg4


100%|██████████| 73034624/73034624 [00:09<00:00, 7515429.45B/s]

2021-10-12 11:49:03,483 copying /tmp/tmpgur_rqg4 to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2021-10-12 11:49:03,585 removing temp file /tmp/tmpgur_rqg4


In [10]:
similarity_by_sent = wieting_sim(args, inputs, preds)
avg_sim_by_sent = similarity_by_sent.mean()
print('\n')
print(avg_sim_by_sent)

Calculating similarity by Wieting subword-embedding SIM model


100%|██████████| 1/1 [00:00<00:00, 18.70it/s]



0.7757033407688141


### Fluency metrics

Additional data preparation:

This part requires CoLa classifier obtained from [here](https://drive.google.com/drive/folders/1p6_3lCbw3J0MhlidvKkRbG73qwmtWuRp). 

IF YOU ARE WORKING IN COLAB: You need to open the link and add the folder `cola_classifier` as an shortcut to your drive (you can allocate shortcut in any place of your drive). The next cell will mount your drive to the current Colab session. You need to specify path of your CoLa shortcut in args `cola_classifier_path`.

IF YOU ARE WORKING ON LOCAL MACHINE: download and unzip the folder from the link and specify the path to the unpacked folder below.

In [29]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("jxuhf/roberta-base-finetuned-cola")

C:\Users\Weizhen Jin\anaconda3\envs\eecs448\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Weizhen Jin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [33]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("jxuhf/roberta-base-finetuned-cola")

In [51]:
import torch
tokens = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
outputs = model(**tokens)
logits = outputs.logits
l = torch.nn.Softmax(dim=1)
cola_score = torch.mean(l(logits)[:, 1])
print(cola_score)

tensor(0.4116, grad_fn=<MeanBackward0>)
